<a href="https://colab.research.google.com/github/amirhoseinaghaei/Research_Simulation/blob/main/Research_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!fusermount -u drive
!google-drive-ocamlfuse drive

fusermount: failed to unmount /content/drive: No such file or directory
/bin/bash: google-drive-ocamlfuse: command not found


In [2]:
from google.colab import drive 
drive.mount('/content/gdrive/') 

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
%cd gdrive/MyDrive/Research_Simmulation/


/content/gdrive/MyDrive/Research_Simmulation


In [ ]:



import numpy as np
# from Environment import Actor
from Rudder import LessonBuffer
from Environment import Environment
from Rudder import RRLSTM as LSTM
import torch
import time as Time
import random
from PolicyUpdater import PolicyUpdater

lb_size = 2048
n_lstm = 16
max_time = 50
policy_lr = 0.5
lstm_lr = 1e-2
l2_regularization = 1e-6
avg_window = 750

Lesson_buffer = LessonBuffer(1000, 25, 5)
episode = 0
rudder_lstm = LSTM(state_input_size=5, n_actions= 2, buffer=Lesson_buffer, n_units=n_lstm,
                        lstm_lr=lstm_lr, l2_regularization=l2_regularization, return_scaling=10,
                        lstm_batch_size=8, continuous_pred_factor=0.5)

rudder_lstm.load_state_dict(torch.load('rudder_lstm_a0.pt'))
environment = Environment(100,25)
environment.CreateStates()
# print(type(environment.StateList[0].Name))
policy_updator  = PolicyUpdater(environment= environment, lr = policy_lr)
episode = 0
visited_dict_0 = {}
visited_dict_1 = {}
for i in range(10000):
    episode += 1
    environment.reset_paramter()
    state = environment.reset_state()
    rewards = []
    states = [state]
    actions = []
    done = False
    name = f'({state[0]}, {state[1]}, {state[2]}, {state[3]}, {state[4]})'

    while not done:
        if episode <= 24:
          action = np.random.choice(2)
        else:
          if np.random.random() < 0.15:
              action = np.random.choice(2)
              if len(states) == 1 and action == 0:
                if name not in visited_dict_0.keys(): 
                    visited_dict_0[name] = 1
                else:
                    visited_dict_0[name] += 1
              if len(states) == 1 and action == 1:
                if name not in visited_dict_1.keys(): 
                    visited_dict_1[name] = 1
                else:
                    visited_dict_1[name] += 1  
      #    
          else:
            # if policy_updator.Quality[name,0] > policy_updator.Quality[name,1] else 1
              action = 0 if policy_updator.Quality[name,0] > policy_updator.Quality[name,1] else 1
              if len(states) == 1 and action == 0:
                if name not in visited_dict_0.keys(): 
                    visited_dict_0[name] = 1
                else:
                    visited_dict_0[name] += 1
              if len(states) == 1 and action == 1:
                if name not in visited_dict_1.keys(): 
                    visited_dict_1[name] = 1
                else:
                    visited_dict_1[name] += 1
        if environment.state.Ra == 0 and environment.state.U == 0:
            action = 0
        if environment.state.Ra == 0 and environment.state.U == 24:
            action = 1
        if environment.state.U > 0:
            action = 0
        state, reward, done = environment.step(action)
    
        actions.append(action)
        states.append(state)
        rewards.append(reward) 
        if done: 

            res = np.nonzero(rewards)[0]
            if len(res) > 0 :
              # print(res)
              rewards[-1] = rewards[res[0]]
              rewards[res[0]] = 0   
            for i in states: 
                if i[1] == "Ch1":
                    i[1] = 1
                else:
                    i[1] = 0
            states = np.stack(states)
            states = states.astype(int)
            rewards = np.array(rewards, dtype = np.float32)
            actions = np.array(actions)
            # Lesson_buffer.add(states = states, actions = actions, rewards = rewards)
            # if  episode < 2000 and Lesson_buffer.full_enough() and Lesson_buffer.different_returns_encountered()  :
            #         # print("different_returns_encountered")        
            #         # If RUDDER is run, the LSTM is trained after each episode until its loss is below a threshold.
            #         # Samples will be drawn from the lessons buffer.
            #         if episode % 25 == 0:

            #             # print("True")
            #             print(episode)
            #             rudder_lstm.train(episode=episode)
            #         if episode >= 1800: 
            #             torch.save(rudder_lstm.state_dict(), 'rudder_lstm.pt')
            #         # Then the LSTM is used to redistribute the reward.
            print(rewards)
            print(states)
            print(actions)
            
            rewards = rudder_lstm.redistribute_reward(states=np.expand_dims(states, 0),actions=np.expand_dims(actions, 0))[0, :]
            # policy_updator.Q_learning(actions= actions , states = states, rewards= rewards)
            print(rewards)

for keys, value in policy_updator.Quality.items():
         initial_StateName = []
         for i in environment.initial_State:
            initial_StateName.append(i.Name) 
         if keys[0] in initial_StateName: 
            print('{:15} {:15} {:15}'.format( keys[0] ,  keys[1], value))
print("******************************************************************")
for keys, value in policy_updator.Quality.items():
         initial_StateName = []
         for i in environment.initial_State:
            initial_StateName.append(i.Name) 
         if keys[0] in initial_StateName: 
            if policy_updator.Quality[keys[0],0] > policy_updator.Quality[keys[0],1]:  
              print('{:15} {:15} {:15}'.format( keys[0] , "Wait" , policy_updator.Quality[keys[0] ,0]))
            else:
              print('{:15} {:15} {:15}'.format( keys[0] , "Send Back" , policy_updator.Quality[keys[0] ,1]))
print("******************************************************************")
for keys, value in policy_updator.Quality.items():
         initial_StateName = []
         for i in environment.initial_State:
            initial_StateName.append(i.Name) 
         if keys[0] in initial_StateName: 
            if policy_updator.Quality[keys[0],0] > 0:  
              print('{:15} {:15} {:15}'.format( keys[0] , "Wait" , policy_updator.Quality[keys[0] ,0]))
            else:
              print('{:15} {:15} {:15}'.format( keys[0] , "Send Back" , policy_updator.Quality[keys[0] ,1]))
print("******************************************************************")
for value in visited_dict_0:
        print('{:15} {:15} {:15}'.format("Wait", value, visited_dict_0[value]))


print("******************************************************************")
for value in visited_dict_1:
        print('{:15} {:15} {:15}'.format("Send Back", value, visited_dict_1[value]))


In [4]:



import numpy as np
# from Environment import Actor
from Rudder import LessonBuffer
from Environment import Environment
from Rudder import RRLSTM as LSTM
import torch
import time as Time
import random
from PolicyUpdater import PolicyUpdater

lb_size = 2048
n_lstm = 16
max_time = 50
policy_lr = 0.5
lstm_lr = 1e-2
l2_regularization = 1e-6
avg_window = 750

Lesson_buffer_a0 = LessonBuffer(1000, 25, 5)
episode = 0
rudder_lstm_a0 = LSTM(state_input_size=5, n_actions= 2, buffer=Lesson_buffer_a0, n_units=n_lstm,
                        lstm_lr=lstm_lr, l2_regularization=l2_regularization, return_scaling=10,
                        lstm_batch_size=8, continuous_pred_factor=0.5)
# rudder_lstm.load_state_dict(torch.load('rudder_lstm.pt'))
environment = Environment(100,25)
environment.CreateStates()
policy_updator  = PolicyUpdater(environment= environment, lr = policy_lr)
episode = 0
for i in range(2050):
    episode += 1
    environment.reset_paramter()
    state = environment.reset_state()
    rewards = []
    states = [state]
    actions = []
    done = False
    name = f'({state[0]}, {state[1]}, {state[2]}, {state[3]}, {state[4]})'

    while not done:
        action = 0 
        if environment.state.Ra == 0 and environment.state.U == 0:
            action = 0
        if environment.state.Ra == 0 and environment.state.U == 24:
            action = 1
        if environment.state.U > 0:
            action = 0
        state, reward, done = environment.step(action)
        actions.append(action)
        states.append(state)
        rewards.append(reward) 
        if done: 

            res = np.nonzero(rewards)[0]
            if len(res) > 0 :
              # print(res)
              rewards[-1] = rewards[res[0]]
              rewards[res[0]] = 0   
            for i in states: 
                if i[1] == "Ch1":
                    i[1] = 1
                else:
                    i[1] = 0
            states = np.stack(states)
            states = states.astype(int)
            rewards = np.array(rewards, dtype = np.float32)
            actions = np.array(actions)
            Lesson_buffer_a0.add(states = states, actions = actions, rewards = rewards)
            if  episode < 2000 and Lesson_buffer_a0.full_enough() and Lesson_buffer_a0.different_returns_encountered()  :
                    if episode % 25 == 0:

                        print(episode)
                        rudder_lstm_a0.train(episode=episode)
                    if episode >= 1800: 
                        torch.save(rudder_lstm_a0.state_dict(), 'rudder_lstm_wait.pt')

            rewards = rudder_lstm_a0.redistribute_reward(states=np.expand_dims(states, 0),actions=np.expand_dims(actions, 0))[0, :]
            policy_updator.Q_learning(actions= actions , states = states, rewards= rewards)

for keys, value in policy_updator.Quality.items():
         initial_StateName = []
         for i in environment.initial_State:
            initial_StateName.append(i.Name) 
         if keys[0] in initial_StateName: 
            print('{:15} {:15} {:15}'.format( keys[0] ,  keys[1], value))
print("******************************************************************")
for keys, value in policy_updator.Quality.items():
         initial_StateName = []
         for i in environment.initial_State:
            initial_StateName.append(i.Name) 
         if keys[0] in initial_StateName: 
            if policy_updator.Quality[keys[0],0] > policy_updator.Quality[keys[0],1]:  
              print('{:15} {:15} {:15}'.format( keys[0] , "Wait" , policy_updator.Quality[keys[0] ,0]))
            else:
              print('{:15} {:15} {:15}'.format( keys[0] , "Send Back" , policy_updator.Quality[keys[0] ,1]))


[[0.65, 0.35], [0.75, 0.25]]
[[0.65, 0.35], [0.75, 0.25]]
Amir
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
975
1000
1025
1050
1075
1100
1125
1150
1175
1200
1225
1250
1275
1300
1325
1350
1375
1400
1425
1450
1475
1500
1525
1550
1575
1600
1625
1650
1675
1700
1725
1750
1775
1800
1825
1850
1875
1900
1925
1950
1975
(31, Ch1, 120, 1, 0)               0 0.13788723945617676
(31, Ch1, 120, 1, 0)               1               0
(31, Ch2, 120, 1, 0)               0 5.588197708129883
(31, Ch2, 120, 1, 0)               1               0
(61, Ch1, 120, 1, 0)               0 0.14683568477630615
(61, Ch1, 120, 1, 0)               1               0
(61, Ch2, 120, 1, 0)               0 7.216609954833984
(61, Ch2, 120, 1, 0)               1               0
(91, Ch1, 120, 1, 0)               0 2.2283217906951904
(91, Ch1, 120, 1, 0)               1               0
(91, Ch2, 120, 1, 0)               0 3.594406843185425
(91, Ch2, 120, 1, 0) 

In [5]:
import numpy as np
# from Environment import Actor
from Rudder import LessonBuffer
from Environment import Environment
from Rudder import RRLSTM as LSTM
import torch
import time as Time
import random
from PolicyUpdater import PolicyUpdater

lb_size = 2048
n_lstm = 16
max_time = 50
policy_lr = 0.5
lstm_lr = 1e-2
l2_regularization = 1e-6
avg_window = 750

Lesson_buffer_a1 = LessonBuffer(1000, 25, 5)
episode = 0

rudder_lstm_a1 = LSTM(state_input_size=5, n_actions= 2, buffer=Lesson_buffer_a1, n_units=n_lstm,
                        lstm_lr=lstm_lr, l2_regularization=l2_regularization, return_scaling=10,
                        lstm_batch_size=8, continuous_pred_factor=0.5)
# rudder_lstm.load_state_dict(torch.load('rudder_lstm.pt'))
environment = Environment(100,25)
environment.CreateStates()
policy_updator  = PolicyUpdater(environment= environment, lr = policy_lr)
episode = 0
visited_dict_0 = {}
for i in range(2050):
    episode += 1
    environment.reset_paramter()
    state = environment.reset_state()
    rewards = []
    states = [state]
    actions = []
    done = False
    name = f'({state[0]}, {state[1]}, {state[2]}, {state[3]}, {state[4]})'

    while not done:
        action = 1 
        if environment.state.Ra == 0 and environment.state.U == 0:
            action = 0
        if environment.state.Ra == 0 and environment.state.U == 24:
            action = 1
        if environment.state.U > 0:
            action = 0
        state, reward, done = environment.step(action)
        actions.append(action)
        states.append(state)
        rewards.append(reward) 
        if done: 

            res = np.nonzero(rewards)[0]
            if len(res) > 0 :
              # print(res)
              rewards[-1] = rewards[res[0]]
              rewards[res[0]] = 0   
            for i in states: 
                if i[1] == "Ch1":
                    i[1] = 1
                else:
                    i[1] = 0
            states = np.stack(states)
            states = states.astype(int)
            rewards = np.array(rewards, dtype = np.float32)
            actions = np.array(actions)
            Lesson_buffer_a1.add(states = states, actions = actions, rewards = rewards)
            if  episode < 2000 and Lesson_buffer_a1.full_enough() and Lesson_buffer_a1.different_returns_encountered()  :
                    if episode % 25 == 0:

                        # print("True")
                        print(episode)
                        rudder_lstm_a1.train(episode=episode)
                    if episode >= 1800: 
                        torch.save(rudder_lstm_a1.state_dict(), 'rudder_lstm_send.pt')
            rewards = rudder_lstm_a1.redistribute_reward(states=np.expand_dims(states, 0),actions=np.expand_dims(actions, 0))[0, :]
            policy_updator.Q_learning(actions= actions , states = states, rewards= rewards)

for keys, value in policy_updator.Quality.items():
         initial_StateName = []
         for i in environment.initial_State:
            initial_StateName.append(i.Name) 
         if keys[0] in initial_StateName: 
            print('{:15} {:15} {:15}'.format( keys[0] ,  keys[1], value))
print("******************************************************************")
for keys, value in policy_updator.Quality.items():
         initial_StateName = []
         for i in environment.initial_State:
            initial_StateName.append(i.Name) 
         if keys[0] in initial_StateName: 
            if policy_updator.Quality[keys[0],0] > policy_updator.Quality[keys[0],1]:  
              print('{:15} {:15} {:15}'.format( keys[0] , "Wait" , policy_updator.Quality[keys[0] ,0]))
            else:
              print('{:15} {:15} {:15}'.format( keys[0] , "Send Back" , policy_updator.Quality[keys[0] ,1]))


[[0.65, 0.35], [0.75, 0.25]]
Amir
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
975
1000
1025
1050
1075
1100
1125
1150
1175
1200
1225
1250
1275
1300
1325
1350
1375
1400
1425
1450
1475
1500
1525
1550
1575
1600
1625
1650
1675
1700
1725
1750
1775
1800
1825
1850
1875
1900
1925
1950
1975
(31, Ch1, 120, 1, 0)               0               0
(31, Ch1, 120, 1, 0)               1 5.267852783203125
(31, Ch2, 120, 1, 0)               0               0
(31, Ch2, 120, 1, 0)               1 1.3568440675735474
(61, Ch1, 120, 1, 0)               0               0
(61, Ch1, 120, 1, 0)               1 4.5850510597229
(61, Ch2, 120, 1, 0)               0               0
(61, Ch2, 120, 1, 0)               1 1.3457705974578857
(91, Ch1, 120, 1, 0)               0               0
(91, Ch1, 120, 1, 0)               1 5.89528751373291
(91, Ch2, 120, 1, 0)               0               0
(91, Ch2, 120, 1, 0)               1 1.3705813884735107
(1

In [12]:



import numpy as np
# from Environment import Actor
from Rudder import LessonBuffer
from Environment import Environment
from Rudder import RRLSTM as LSTM
import torch
import time as Time
import random
from PolicyUpdater import PolicyUpdater

lb_size = 2048
n_lstm = 16
max_time = 50
policy_lr = 0.5
lstm_lr = 1e-2
l2_regularization = 1e-6
avg_window = 750

episode = 0


rudder_lstm_a0.load_state_dict(torch.load('rudder_lstm_a00.pt'))
rudder_lstm_a1.load_state_dict(torch.load('rudder_lstm_a11.pt'))

environment = Environment(100,25)
environment.CreateStates()
policy_updator  = PolicyUpdater(environment= environment, lr = policy_lr)
episode = 0
for i in range(5000):
    episode += 1
    environment.reset_paramter()
    state = environment.reset_state()
    rewards = []
    states = [state]
    actions = []
    done = False
    name = f'({state[0]}, {state[1]}, {state[2]}, {state[3]}, {state[4]})'

    while not done:
        if episode <= 24:
          action = np.random.choice(2)
        else:
          if np.random.random() < 0.15:
              action = np.random.choice(2) 
          else:
              action = 0 if policy_updator.Quality[name,0] > policy_updator.Quality[name,1] else 1
        if environment.state.Ra == 0 and environment.state.U == 0:
            action = 0
        if environment.state.Ra == 0 and environment.state.U == 24:
            action = 1
        if environment.state.U > 0:
            action = 0
        state, reward, done = environment.step(action)
    
        actions.append(action)
        states.append(state)
        rewards.append(reward) 
        if done: 

            res = np.nonzero(rewards)[0]
            if len(res) > 0 :
              # print(res)
              rewards[-1] = rewards[res[0]]
              rewards[res[0]] = 0   
            for i in states: 
                if i[1] == "Ch1":
                    i[1] = 1
                else:
                    i[1] = 0
            states = np.stack(states)
            states = states.astype(int)
            rewards = np.array(rewards, dtype = np.float32)
            actions = np.array(actions)
            if actions[0] == 0: 
              rewards = rudder_lstm_a0.redistribute_reward(states=np.expand_dims(states, 0),actions=np.expand_dims(actions, 0))[0, :]
            if actions[0] == 1: 
              rewards = rudder_lstm_a1.redistribute_reward(states=np.expand_dims(states, 0),actions=np.expand_dims(actions, 0))[0, :]
            
            policy_updator.Q_learning(actions= actions , states = states, rewards= rewards)

for keys, value in policy_updator.Quality.items():
         initial_StateName = []
         for i in environment.initial_State:
            initial_StateName.append(i.Name) 
         if keys[0] in initial_StateName: 
            print('{:15} {:15} {:15}'.format( keys[0] ,  keys[1], value))
print("******************************************************************")
for keys, value in policy_updator.Quality.items():
         initial_StateName = []
         for i in environment.initial_State:
            initial_StateName.append(i.Name) 
         if keys[0] in initial_StateName: 
            if policy_updator.Quality[keys[0],0] > policy_updator.Quality[keys[0],1]:  
              print('{:15} {:15} {:15}'.format( keys[0] , "Wait" , policy_updator.Quality[keys[0] ,0]))
            else:
              print('{:15} {:15} {:15}'.format( keys[0] , "Send Back" , policy_updator.Quality[keys[0] ,1]))

[[0.65, 0.35], [0.75, 0.25]]
Amir
(31, Ch1, 120, 1, 0)               0 -2.8724098205566406
(31, Ch1, 120, 1, 0)               1 7.914113998413086
(31, Ch2, 120, 1, 0)               0 3.197254180908203
(31, Ch2, 120, 1, 0)               1 1.4092371463775635
(61, Ch1, 120, 1, 0)               0 -2.51678729057312
(61, Ch1, 120, 1, 0)               1 6.641089916229248
(61, Ch2, 120, 1, 0)               0 4.642870903015137
(61, Ch2, 120, 1, 0)               1 1.006040096282959
(91, Ch1, 120, 1, 0)               0 -3.2509591579437256
(91, Ch1, 120, 1, 0)               1 2.3370614051818848
(91, Ch2, 120, 1, 0)               0 8.157583236694336
(91, Ch2, 120, 1, 0)               1 -1.3419480323791504
(121, Ch1, 120, 1, 0)               0 -0.24785232543945312
(121, Ch1, 120, 1, 0)               1 5.701873779296875
(121, Ch2, 120, 1, 0)               0 5.698758125305176
(121, Ch2, 120, 1, 0)               1 -0.8140076994895935
******************************************************************
(3

 **Finding Optimal policies with DQN** 

In [6]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np


class DeepQNetwork(nn.Module):
    def __init__(self, lr, input_dims, fc1_dims, fc2_dims,
                 n_actions):
        super(DeepQNetwork, self).__init__()
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.fc1 = nn.Linear(*self.input_dims, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.fc3 = nn.Linear(self.fc2_dims, self.n_actions)

        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, state):
        state = T.Tensor(state).to(self.device)
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        actions = self.fc3(x)

        return actions


class Agent:
    def __init__(self, gamma, epsilon, lr, input_dims, batch_size, n_actions,
                 max_mem_size=100000, eps_end=0.05, eps_dec=5e-4):
        self.gamma = gamma
        self.epsilon = epsilon
        self.eps_min = eps_end
        self.eps_dec = eps_dec
        self.lr = lr
        self.action_space = [i for i in range(n_actions)]
        self.mem_size = max_mem_size
        self.batch_size = batch_size
        self.mem_cntr = 0
        self.iter_cntr = 0
        self.replace_target = 100

        self.Q_eval = DeepQNetwork(lr, n_actions=n_actions,
                                   input_dims=input_dims,
                                   fc1_dims=256, fc2_dims=256)
        self.state_memory = np.zeros((self.mem_size, *input_dims),
                                     dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, *input_dims),
                                         dtype=np.float32)
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.bool)

    def store_transition(self, state, action, reward, state_, terminal):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.reward_memory[index] = reward
        self.action_memory[index] = action
        self.terminal_memory[index] = terminal

        self.mem_cntr += 1

    def choose_action(self, observation):
        if np.random.random() > self.epsilon:
            # state = T.tensor(observation).to(self.Q_eval.device)      
            actions = self.Q_eval.forward(observation)
            action = T.argmax(actions).item()
        else:
            action = np.random.choice(self.action_space)
        return action

    def learn(self):
        if self.mem_cntr < self.batch_size:
            return

        self.Q_eval.optimizer.zero_grad()

        max_mem = min(self.mem_cntr, self.mem_size)

        batch = np.random.choice(max_mem, self.batch_size, replace=False)
        batch_index = np.arange(self.batch_size, dtype=np.int32)

        state_batch = T.tensor(self.state_memory[batch]).to(self.Q_eval.device)
        new_state_batch = T.tensor(
                self.new_state_memory[batch]).to(self.Q_eval.device)
        action_batch = self.action_memory[batch]
        reward_batch = T.tensor(
                self.reward_memory[batch]).to(self.Q_eval.device)
        terminal_batch = T.tensor(
                self.terminal_memory[batch]).to(self.Q_eval.device)

        q_eval = self.Q_eval.forward(state_batch)[batch_index, action_batch]
        q_next = self.Q_eval.forward(new_state_batch)
        q_next[terminal_batch] = 0.0

        q_target = reward_batch + self.gamma*T.max(q_next, dim=1)[0]

        loss = self.Q_eval.loss(q_target, q_eval).to(self.Q_eval.device)
        loss.backward()
        self.Q_eval.optimizer.step()

        self.iter_cntr += 1
        self.epsilon = self.epsilon - self.eps_dec \
            if self.epsilon > self.eps_min else self.eps_min

In [10]:

import numpy as np
# from Environment import Actor
from Rudder import LessonBuffer
from Environment import Environment
from Rudder import RRLSTM as LSTM
import torch
import time as Time
import random
from PolicyUpdater import PolicyUpdater
from tqdm import tqdm
lb_size = 2048
n_lstm = 16
max_time = 50
policy_lr = 0.5
lstm_lr = 1e-2
l2_regularization = 1e-6
avg_window = 750

episode = 0


rudder_lstm_a0.load_state_dict(torch.load('rudder_lstm_wait.pt'))
rudder_lstm_a1.load_state_dict(torch.load('rudder_lstm_send.pt'))


DQN_Agent = Agent(gamma=0.99, epsilon=1.0, batch_size=64, n_actions=2, eps_end=0.01,
                  input_dims=[5], lr=0.001)

environment = Environment(100,25)
environment.CreateStates()
policy_updator  = PolicyUpdater(environment= environment, lr = policy_lr)
episode = 0
for i in tqdm(range(2000)):
    # print(episode)
    episode += 1
    environment.reset_paramter()
    state = environment.reset_state()
    rewards = []
    states = [state]
    actions = []
    dones = []
    done = False
    name = f'({state[0]}, {state[1]}, {state[2]}, {state[3]}, {state[4]})'
    # prev_state = state

    initial_state= state
    if initial_state[1] == "Ch1":
        initial_state[1] = 1
    else:
        initial_state[1] = 0
    initial_state = initial_state.astype(int)

    while not done:
        if episode <= 50:
          action = np.random.choice(2)
        else:
          if np.random.random() < 0.15:
              action = np.random.choice(2) 
          else:      
              action = DQN_Agent.choose_action(initial_state)
        if environment.state.Ra == 0 and environment.state.U == 0:
            action = 0
        if environment.state.Ra == 0 and environment.state.U == 24:
            action = 1
        if environment.state.U > 0:
            action = 0
        state, reward, done = environment.step(action)

        # prev_state = state
        actions.append(action)
        states.append(state)
        rewards.append(reward) 
        dones.append(done)
        if done: 

            res = np.nonzero(rewards)[0]
            if len(res) > 0 :
              # print(res)
              rewards[-1] = rewards[res[0]]
              rewards[res[0]] = 0   
            for i in states: 
                if i[1] == "Ch1":
                    i[1] = 1
                else:
                    i[1] = 0
            states = np.stack(states)
            states = states.astype(int)
            rewards = np.array(rewards, dtype = np.float32)
            actions = np.array(actions)
            if actions[0] == 0: 
              rewards = rudder_lstm_a0.redistribute_reward(states=np.expand_dims(states, 0),actions=np.expand_dims(actions, 0))[0, :]
            if actions[0] == 1: 
              rewards = rudder_lstm_a1.redistribute_reward(states=np.expand_dims(states, 0),actions=np.expand_dims(actions, 0))[0, :]
            # print(dones)
            # print(states)
            for i in range(len(states) -1 ):
              DQN_Agent.store_transition(states[i], actions[i], rewards[i], states[i+1],
                                  dones[i])
            DQN_Agent.learn()
            # print(DQN_Agent.action_memory[0:25])
            # print(DQN_Agent.state_memory[0:25])
            # print(DQN_Agent.reward_memory[0:25])
            # print(DQN_Agent.new_state_memory[0:25])
            # print(len(DQN_Agent.action_memory))
            # print(len(DQN_Agent.new_state_memory))
            # Time.sleep(10)

for state in environment.initial_State:
  State  =  np.array([state.Au, state.Ch, state.BT, state.Ra, state.U])
  if State[1] == "Ch1":
    State[1] = 1
  else:
    State[1] = 0
  State = State.astype(int) 
  print(f'{State} ___________________________ {DQN_Agent.choose_action(State)}')
# for keys, value in policy_updator.Quality.items():
#          initial_StateName = []
#          for i in environment.initial_State:
#             initial_StateName.append(i.Name) 
#          if keys[0] in initial_StateName: 
#             print('{:15} {:15} {:15}'.format( keys[0] ,  keys[1], value))
# print("******************************************************************")
# for keys, value in policy_updator.Quality.items():
#          initial_StateName = []
#          for i in environment.initial_State:
#             initial_StateName.append(i.Name) 
#          if keys[0] in initial_StateName: 
#             if policy_updator.Quality[keys[0],0] > policy_updator.Quality[keys[0],1]:  
#               print('{:15} {:15} {:15}'.format( keys[0] , "Wait" , policy_updator.Quality[keys[0] ,0]))
#             else:
#               print('{:15} {:15} {:15}'.format( keys[0] , "Send Back" , policy_updator.Quality[keys[0] ,1]))

<ipython-input-6-9bd787e1c7fb>:58: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.terminal_memory = np.zeros(self.mem_size, dtype=np.bool)


[[0.65, 0.35], [0.75, 0.25]]
Amir


100%|██████████| 2000/2000 [00:33<00:00, 59.59it/s]

[ 31   1 120   1   0] ___________________________ 0
[ 31   0 120   1   0] ___________________________ 0
[ 61   1 120   1   0] ___________________________ 0
[ 61   0 120   1   0] ___________________________ 1
[ 91   1 120   1   0] ___________________________ 0
[ 91   0 120   1   0] ___________________________ 1
[121   1 120   1   0] ___________________________ 0
[121   0 120   1   0] ___________________________ 0
